# Exercise: Implementing a Sudoku solver and testing for uniqueness

<b>Goal:</b> In this exercise, we implement a Sudoku solver using an integer program and Python-MIP. As an extra result, we will also see how such a solver can be used to test uniqueness of a solution to a given Sudoku puzzle.

---

## Solving a Sudoku puzzle: Modeling with integer variables

As you most likely all know, a Sudoku is puzzle where the goal is to fill in the cells of a 9x9 grid with integers from $1$ to $9$ such that
- there is precisely one number per cell,
- no row contains two equal numbers,
- no column contains two equal numbers, and
- no one of the nine 3x3 squares that the grid can be partitioned in contains two equal numbers.

An example of a Sudoku puzzle is given below.

<div style="background-color:white">
<center>
    <img src="sudoku_solver_example.png", style="padding-top: 10px;">
</center>
</div>

To solve such a Sudoku puzzle, we have to decide which numbers to assign to which cell. These decisions can be easily modelled by integral decision variables $x_{ijk}$ for $i,j,k\in\{1,\ldots,9\}$ such that

$$
x_{ijk} = \begin{cases} 1 & \text{if cell $(i,j)$ of the Sudoku contains number $k$}\\ 0 & \text{else} \end{cases}\enspace. 
$$

The question is how to set up suitable constraints that guarantee that a feasible $\{0,1\}$-point $x$ does indeed correspond to a solution of the Sudoku.


<b>Your first task:</b> Come up with linear constraints in the variables $x_{ijk}$ that model the conditions imposed on a valid Sudoku solution, i.e., make sure that any $\{0,1\}$-solution of your system corresponds to a feasible solution of a given Sudoku.

#### We use zero-based indexing for grid cells so that translation to code is straightforward.

\begin{align*}
x_{ijk} &= 1 \text{ for all cells $(i, j)$ of the Sudoku that already contain the number $k$} \\
\sum_{k = 1}^9 x_{ijk} &= 1 \text{ for all $i, j \in \{0, \dots, 8\}$} \\
\sum_{j = 0}^8 x_{ijk} &= 1 \text{ for all $i \in \{0, \dots, 8\}$ and $k \in [9]$} \\
\sum_{i = 0}^8 x_{ijk} &= 1 \text{ for all $j \in \{0, \dots, 8\}$ and $k \in [9]$} \\
\sum_{i = 3r}^{3r + 2} \sum_{j = 3c}^{3c + 2} x_{ijk} &= 1 \text{ for all $r, c \in \{0, 1, 2\}$ and $k \in [9]$} \\
x_{ijk} &\in \{0, 1\} \text{ for all $i, j \in \{0, \dots, 8\}$ and $k \in [9]$}
\end{align*}

---

## Implementing integer programs in Python-MIP

Implementing integer programs in Python-MIP is almost the same as implementing linear programs - except that you'll have to declare that you want to put integrality conditions on your variables. Check out the simple IP below.

In [1]:
import mip

simpleProblem = mip.Model(name="Simple IP example", sense=mip.MINIMIZE)

x = simpleProblem.add_var(name="x", var_type=mip.INTEGER)

simpleProblem.objective = x

simpleProblem += x >= 4.5

simpleProblem.optimize()

Welcome to the CBC MILP Solver 
Version: Trunk
Build Date: Oct 24 2021 

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-1) rows, 0 (-1) columns and 0 (-1) elements
Clp0000I Optimal - objective value 4.5
Coin0511I After Postsolve, objective 4.5, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 4.5 - 0 iterations time 0.002, Presolve 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00



<OptimizationStatus.OPTIMAL: 0>

In [2]:
print(x.x)

5.0


As you can see, the `add_var` takes an additional (optional) argument `var_type`, which we can set to `INTEGER`, `BINARY` or `CONTINUOUS` (the latter one being the default). Thus, for the Sudoku problem above, you might want to use binary variables.

<b>Your second task:</b> Implement the constraints that you came up with in the first task in an integer program, and use it to find a solution to an input Sudoku problem. Observe that this is a pure feasibility problem, so you can use an IP with a constant objective.

To this end, you can assume that the Sudoku is given to you as a list of $81$ values, each representing a cell of the Sudoku read row by row from left to right; where a $0$ indicates an empty cell. An example is given below. Note that there also is a function to display a Sudoku that is given in the above form.

Make sure that your function returns the Sudoku in the same format as the input.

In [4]:
# Example Sudoku input and Sudoku printing

sudoku1 = [4, 0, 7, 0, 0, 0, 0, 0, 0, 
           0, 3, 5, 0, 9, 7, 4, 0, 0, 
           0, 9, 0, 0, 0, 0, 0, 0, 6, 
           0, 0, 0, 3, 0, 2, 0, 0, 0, 
           6, 0, 0, 0, 8, 0, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 5, 0, 0, 
           0, 0, 0, 4, 0, 0, 0, 1, 8, 
           0, 0, 3, 0, 2, 8, 0, 0, 4, 
           5, 0, 4, 0, 0, 0, 0, 9, 7]

def printSudoku(sudoku):
    # compact Sudoku printing function
    # taken from https://codegolf.stackexchange.com/questions/126930/
    #    draw-a-sudoku-board-using-line-drawing-characters
    q = lambda x,y:x+y+x+y+x
    r = lambda a,b,c,d,e:a+q(q(b*3,c),d)+e+"\n"
    print(((r(*"╔═╤╦╗") + q(q("║ %d │ %d │ %d "*3 + "║\n",r(*"╟─┼╫╢")), r(*"╠═╪╬╣")) +
            r(*"╚═╧╩╝")) % tuple(sudoku)).replace(*"0 "))

printSudoku(sudoku1)

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │   │ 7 ║   │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 3 │ 5 ║   │ 9 │ 7 ║ 4 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 9 │   ║   │   │   ║   │   │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 3 │   │ 2 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │   │   ║   │ 8 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 5 │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║ 4 │   │   ║   │ 1 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 2 │ 8 ║   │   │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │ 4 ║   │   │   ║   │ 9 │ 7 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [28]:
# Implementation of a Sudoku solver

def sudokuSolver(inputSudoku):
    
    ## Your code goes here.
    sudoku_lp = mip.Model(name="sudoku", sense=mip.MINIMIZE)
    vars = [[[sudoku_lp.add_var(name=f"x_{i}{j}{k}", var_type=mip.BINARY) for k in range(1, 10)] for j in range(9)] for i in range(9)] # create variables

    # third index is one-based, use this function for easier access
    def gv(i, j, k):
        return vars[i][j][k - 1]
    
    # already occupied cells
    for i in range(9):
        for j in range(9):
            k = inputSudoku[9 * i + j]
            if k > 0:
                sudoku_lp += gv(i, j, k) == 1.0
    
    # every cell contains exactly one number 
    for i in range(9):
        for j in range(9):
            sudoku_lp += mip.xsum(gv(i, j, k) for k in range(1, 10)) == 1

    # every row contains every number exactly once
    for i in range(9):
        for k in range(1, 10):
            sudoku_lp += mip.xsum(gv(i, j, k) for j in range(9)) == 1
    
    # every column contains every number exactly once
    for j in range(9):
        for k in range(1, 10):
            sudoku_lp += mip.xsum(gv(i, j, k) for i in range(9)) == 1

    # every 3x3 subgrid contains every number exactly once
    for r in range(3):
        for c in range(3):
            for k in range(1, 10):
                sudoku_lp += mip.xsum(gv(i, j, k) for i in range(3 * r, 3 * r + 3) for j in range(3 * c, 3 * c + 3)) == 1

    sudoku_lp.optimize()
    if sudoku_lp.status == mip.OptimizationStatus.INFEASIBLE:
        return (None, sudoku_lp, vars)

    outputSudoku = []

    # translate IP output back to sudoku
    for i in range(9):
        for j in range(9):
            for k in range(1, 10):
                if gv(i, j, k).x == 1.0:
                    outputSudoku.append(k)
                    break

    return (outputSudoku, sudoku_lp, vars)

In [29]:
printSudoku(sudokuSolver(sudoku1)[0])

Starting solution of the Linear programming relaxation problem using Primal Simplex

Coin0506I Presolve 0 (-349) rows, 0 (-729) columns and 0 (-2941) elements
Clp0000I Optimal - objective value 0
Coin0511I After Postsolve, objective 0, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 0 - 0 iterations time 0.002, Presolve 0.00, Idiot 0.00

Starting MIP optimization
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elements
Cgl0015I Clique Strengthening extended 0 cliques, 0 were dominated
Cbc3007W No integer variables
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.00

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 4 │ 6 │ 7 ║ 2 │ 1 │ 3 ║ 9 │ 8 │ 5 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │ 3 │ 5 ║ 6 │ 9 │ 7 ║ 4 │ 2 │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │ 9 │ 2 ║ 8 │ 4 │ 5 ║ 7 │ 3 │ 6 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 7 │ 5 │ 1 ║ 3 │ 6 │ 2 ║ 8 │ 4 │ 9 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 6 │ 2

---

## Checking for uniqueness of the Sudoku solutions

Sudokus are generally agreed to only be "real" Sudokus if they have a unique Solution.

<b>Your third task:</b> Implement a function that checks whether a Sudoko has no solution, a unique solution, or more than one solution! You can reuse the code that you generated for the Sudoku solver above. The function should return a tuple `(n, sol)`, where $n\in\{0, 1, 2\}$ depending on whether the Sudoku has zero, one, or at least two solutions, respectively, and `sol` is a list of zero, one, or two solutions of the Sudoku.

If you want a hint, run the following code cell. Do not run it if you want to think about the problem yourself! :)

In [38]:
## Running this cell will display a hint!

encoded = [79, 98, 115, 101, 114, 118, 101, 32, 116, 104, 97, 116, 32, 115, 111, 108, 118, 105, 110, 103, 32, 97, 32, 83, 117, 100, 111, 107, 117, 32, 100, 105, 100, 32, 110, 111, 116, 32, 117, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 102, 117, 110, 99, 116, 105, 111, 110, 32, 111, 102, 32, 116, 104, 101, 32, 73, 80, 46, 32, 79, 110, 99, 101, 32, 121, 111, 117, 32, 102, 111, 117, 110, 100, 32, 111, 110, 101, 32, 115, 111, 108, 117, 116, 105, 111, 110, 44, 32, 99, 97, 110, 32, 121, 111, 117, 32, 101, 120, 112, 108, 111, 105, 116, 32, 116, 104, 101, 32, 102, 97, 99, 116, 32, 116, 104, 97, 116, 32, 121, 111, 117, 32, 99, 97, 110, 32, 99, 104, 111, 111, 115, 101, 32, 116, 104, 101, 32, 111, 98, 106, 101, 99, 116, 105, 118, 101, 32, 116, 111, 32, 115, 101, 101, 32, 105, 102, 32, 121, 111, 117, 32, 99, 97, 110, 32, 102, 105, 110, 100, 32, 97, 110, 111, 116, 104, 101, 114, 32, 115, 111, 108, 117, 116, 105, 111, 110, 63]
print('Hint: ' + ''.join([chr(x) for x in encoded]))

Hint: Observe that solving a Sudoku did not use the objective function of the IP. Once you found one solution, can you exploit the fact that you can choose the objective to see if you can find another solution?


In [ ]:
def numberOfSolutions(inputSudoku):
    
    ## Your code goes here.
    sol1, lp, vars = sudokuSolver(inputSudoku)
    if not sol1:
        return (0, [])
    
    # third index is one-based, use this function for easier access
    def gv(i, j, k):
        return vars[i][j][k - 1]

    # add all variables that are set to one in sol1 to current objective, there are 81 such variables
    lp += mip.xsum(gv(i, j, sol1[9 * i + j]) for i in range(9) for j in range(9))

    lp.optimize()

    # try to minimize the above sum, if the minimum is less than 81, another solution is found, otherwise sol1 is the unique solution
    if lp.objective_value == 9 * 9:
        return (1, [sol1])

    # construct the newly found solution
    sol2 = []

    # translate IP output back to sudoku
    for i in range(9):
        for j in range(9):
            for k in range(1, 10):
                if gv(i, j, k).x == 1.0:
                    sol2.append(k)
                    break
    
    return (2, [sol1, sol2])

---

## Testing your code

Among the following three Sudokus, there is one from each category that your function `numberOfSolutions()` should be able to distinguish: One has no solution, one has a unique Solution, and one has two Solutions. Test your implementation on these Sudokus!

In [ ]:
sudoku2 = [2, 0, 0, 0, 0, 0, 0, 4, 0, 
           1, 0, 0, 0, 0, 0, 0, 0, 7,
           8, 0, 6, 3, 0, 0, 0, 0, 0,
           0, 5, 0, 0, 0, 7, 3, 0, 1, 
           0, 0, 3, 0, 1, 0, 0, 0, 0, 
           0, 0, 2, 0, 0, 3, 7, 5, 4, 
           0, 0, 7, 0, 0, 5, 0, 0, 0, 
           5, 0, 0, 0, 4, 0, 0, 0, 0, 
           0, 0, 0, 1, 7, 0, 0, 0, 8]
printSudoku(sudoku2)
# 0 solutions

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║ 2 │   │   ║   │   │   ║   │ 4 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║   │   │   ║   │   │ 7 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 8 │   │ 6 ║ 3 │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │ 5 │   ║   │   │ 7 ║ 3 │   │ 1 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 3 ║   │ 1 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 2 ║   │   │ 3 ║ 7 │ 5 │ 4 ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │ 7 ║   │   │ 5 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 5 │   │   ║   │ 4 │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 1 │ 7 │   ║   │   │ 8 ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [ ]:
sudoku3 = [0, 0, 0, 6, 0, 7, 0, 0, 0, 
           0, 0, 0, 0, 0, 0, 0, 9, 8,
           3, 0, 0, 0, 0, 0, 0, 0, 0,
           0, 0, 0, 0, 2, 0, 6, 0, 0, 
           0, 0, 0, 0, 0, 0, 7, 0, 0, 
           0, 4, 0, 0, 8, 0, 0, 0, 0, 
           1, 0, 0, 0, 0, 0, 0, 2, 3, 
           0, 0, 8, 9, 0, 0, 0, 0, 0, 
           0, 0, 0, 4, 0, 0, 1, 0, 0]
printSudoku(sudoku3)
# 1 solution

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │   │   ║ 6 │   │ 7 ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║   │ 9 │ 8 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 2 │   ║ 6 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║   │   │   ║ 7 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │ 4 │   ║   │ 8 │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║ 1 │   │   ║   │   │   ║   │ 2 │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 8 ║ 9 │   │   ║   │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │   ║ 4 │   │   ║ 1 │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [ ]:
sudoku4 = [0, 6, 0, 0, 0, 0, 0, 7, 4,
           1, 0, 0, 6, 0, 7, 0, 0, 3, 
           7, 0, 0, 0, 0, 0, 0, 0, 0, 
           0, 0, 0, 0, 1, 0, 0, 0, 2, 
           0, 0, 1, 5, 0, 0, 9, 0, 0, 
           9, 0, 0, 8, 0, 0, 0, 1, 0, 
           0, 0, 0, 0, 0, 0, 0, 3, 0, 
           3, 0, 0, 0, 0, 2, 8, 5, 0, 
           0, 0, 9, 0, 0, 4, 0, 0, 0]
printSudoku(sudoku4)
# 2 solutions

╔═══╤═══╤═══╦═══╤═══╤═══╦═══╤═══╤═══╗
║   │ 6 │   ║   │   │   ║   │ 7 │ 4 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 1 │   │   ║ 6 │   │ 7 ║   │   │ 3 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 7 │   │   ║   │   │   ║   │   │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │ 1 │   ║   │   │ 2 ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 1 ║ 5 │   │   ║ 9 │   │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 9 │   │   ║ 8 │   │   ║   │ 1 │   ║
╠═══╪═══╪═══╬═══╪═══╪═══╬═══╪═══╪═══╣
║   │   │   ║   │   │   ║   │ 3 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║ 3 │   │   ║   │   │ 2 ║ 8 │ 5 │   ║
╟───┼───┼───╫───┼───┼───╫───┼───┼───╢
║   │   │ 9 ║   │   │ 4 ║   │   │   ║
╚═══╧═══╧═══╩═══╧═══╧═══╩═══╧═══╧═══╝



In [ ]:
# Test your functions here!
def printResults(inputSudoku):
  n, res = numberOfSolutions(inputSudoku)
  print()
  print(f"{n} solutions found.")
  for q in res:
    printSudoku(q)

printResults(sudoku4)